In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op, centralize, dsift, sparse_hog
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.aam import (
    AAMBuilder, PatchAAMBuilder, LinearAAMBuilder, LinearPatchAAMBuilder, PartsAAMBuilder,
    CRAAMFitter, PSD, PAJ)
from menpofit.aam.algorithm.cr2 import BAJ

from menpofit.visualize import visualize_aam, visualize_fitting_result

In [ ]:
from functools import partial

fast_dsift2 = partial(dsift, fast=True, window_size=5, geometry=(1, 1, 12))
fast_dsift2.__name__ = 'fast_dsift'

## Load training data

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/',  
                           verbose=True, max_images=None):
    i.rescale_landmarks_to_diagonal_range(200)
    i.crop_to_landmarks_proportion_inplace(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

In [ ]:
visualize_images(training_images)

## Load test data

In [ ]:
test_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/testset/', 
                           verbose=True, max_images=None):    
    i.rescale_landmarks_to_diagonal_range(200)
    i.crop_to_landmarks_proportion_inplace(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    test_images.append(i)

In [ ]:
visualize_images(test_images)

## Build Active Appearance Models

In [ ]:
features = sparse_hog
diagonal = 200
scales = (1, .5)
scale_shapes = False
scale_features = True
max_shape_components= 25
max_appearance_components = None

patch_shape = (17, 17)

normalize_parts = no_op

#### AAM

In [ ]:
aam_builder = AAMBuilder(
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

aam = aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

In [ ]:
from menpofit.visualize import visualize_appearance_model

visualize_appearance_model(aam.appearance_models[0])

#### Patch based AAM

In [ ]:
patch_aam_builder = PatchAAMBuilder(
    patch_shape=patch_shape,
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

patch_aam = patch_aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

#### Linear AAM

In [ ]:
linear_aam_builder = LinearAAMBuilder(
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

linear_aam = linear_aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

#### Linear Patch based AAM

In [ ]:
linear_patch_aam_builder = LinearPatchAAMBuilder(
    patch_shape=patch_shape,
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

linear_patch_aam = linear_patch_aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

#### Parts based AAM

In [ ]:
parts_aam_builder = PartsAAMBuilder(
    patch_shape=patch_shape, 
    features=features,
    normalize_parts=normalize_parts,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

parts_aam = parts_aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

## Fit Active Appearance Models

In [ ]:
cr_algorithm_cls = BAJ
n_shape = [5, 15]
n_appearance = [50, 100]
n_perturbations = 25
sampling_step = [8, 8]

max_iters = 12
gamma = 1

#### AAM Fitter

In [ ]:
aam_fitter = CRAAMFitter(
    aam, 
    cr_algorithm_cls=cr_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    n_perturbations=n_perturbations,
    max_iters=max_iters,
    sampling=sampling_step)

aam_fitter.train(
    training_images, 
    group='ibug_face_66', 
    gamma=gamma,
    verbose=True)

#### Patch based AAM Fitter

In [ ]:
patch_aam_fitter = CRAAMFitter(
    patch_aam, 
    cr_algorithm_cls=cr_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    n_perturbations=n_perturbations,
    max_iters=max_iters,
    sampling=sampling_step)

patch_aam_fitter.train(
    training_images, 
    group='ibug_face_66', 
    gamma=gamma,
    verbose=True)

#### Linear AAM Fitter

In [ ]:
linear_aam_fitter = CRAAMFitter(
    linear_aam, 
    cr_algorithm_cls=cr_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    n_perturbations=n_perturbations,
    max_iters=max_iters,
    sampling=sampling_step)

linear_aam_fitter.train(
    training_images, 
    group='ibug_face_66', 
    gamma=gamma,
    verbose=True)

#### Linear Patch based AAM Fitter

In [ ]:
linear_patch_aam_fitter = CRAAMFitter(
    linear_patch_aam, 
    cr_algorithm_cls=cr_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    n_perturbations=n_perturbations,
    max_iters=max_iters,
    sampling=sampling_step)

linear_patch_aam_fitter.train(
    training_images, 
    group='ibug_face_66', 
    gamma=gamma,
    verbose=True)

#### Parts based AAM Fitter

In [ ]:
sampling_mask = parts_mask = np.require(np.zeros(patch_shape, dtype=np.bool))
sampling_mask[2::6, 2::6] = True

imshow(sampling_mask)

In [ ]:
parts_aam_fitter = CRAAMFitter(
    parts_aam, 
    cr_algorithm_cls=cr_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    n_perturbations=n_perturbations,
    max_iters=max_iters,
    sampling=sampling_mask)

parts_aam_fitter.train(
    training_images, 
    group='ibug_face_66', 
    gamma=gamma,
    verbose=True)

#### Fitting loop

In [ ]:
#np.random.seed(seed=1)

fitters = [#aam_fitter,
           #patch_aam_fitter,
           #linear_aam_fitter,
           #linear_patch_aam_fitter,
           parts_aam_fitter]

fitters_results =  [[], []]

for j, i in enumerate(test_images):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitters[0].get_initial_shape_from_shape(gt_s, noise_std=0.04)

    for k, fitter in enumerate(fitters):
        
        fr = fitter.fit(i, s, gt_shape=gt_s)
        fr.downscale = 0.5   
        fitters_results[k].append(fr)
    
        print 'Image: ', j
        print fr

## Visualize Results

In [ ]:
visualize_fitting_result(fitters_results[-1])

In [ ]:
visualize_fitting_result(fitters_results[0])